In [ ]:
import json
import os
import shutil

def round_to_half(score):
    """Round the float to the nearest half (.0 or .5)"""
    return round(score * 2) / 2

# Directory where the images are stored
image_dir = 'path_to_your_images_directory'

# Read the content of the text file
with open("your_file.txt", "r") as file:
    lines = file.readlines()

# Process each line to extract filename and score
for line in lines:
    filename, score = line.strip().split()
    rounded_score = round_to_half(float(score))  # Apply rounding to nearest half

    # Create the score folder if it doesn't exist
    score_folder = os.path.join(image_dir, str(rounded_score))
    if not os.path.exists(score_folder):
        os.makedirs(score_folder)
    
    # Create 'image' and 'output' subfolders
    image_subfolder = os.path.join(score_folder, 'image')
    output_subfolder = os.path.join(score_folder, 'output')

    if not os.path.exists(image_subfolder):
        os.makedirs(image_subfolder)
    if not os.path.exists(output_subfolder):
        os.makedirs(output_subfolder)

    # Move the image file into the 'image' subfolder
    image_path = os.path.join(image_dir, filename)
    new_image_path = os.path.join(image_subfolder, filename)

    if os.path.exists(image_path):
        shutil.move(image_path, new_image_path)
    else:
        print(f"Image {filename} not found!")

    # Create JSON output for this file
    json_data = {"filename": filename, "score": rounded_score}
    json_output_path = os.path.join(output_subfolder, f"{filename}.json")

    with open(json_output_path, "w") as json_file:
        json.dump(json_data, json_file, indent=4)

print("Processing complete.")


In [9]:
import json
import os
import shutil

image_dir = 'Images'

# Read the content of the text file
with open("label.txt", "r") as file:
    lines = file.readlines()

# Process each line to extract filename and score
for line in lines:
    filename, score = line.strip().split()
    rounded_score = int(float(score))  # Apply rounding to nearest half

    # Create the score folder if it doesn't exist
    score_folder = os.path.join(image_dir, str(rounded_score))
    if not os.path.exists(score_folder):
        os.makedirs(score_folder)
    
    # Create 'image' and 'output' subfolders
    image_subfolder = score_folder

    if not os.path.exists(image_subfolder):
        os.makedirs(image_subfolder)


    # Move the image file into the 'image' subfolder
    image_path = os.path.join(image_dir, filename)
    new_image_path = os.path.join(image_subfolder, filename)

    if os.path.exists(image_path):
        shutil.copy(image_path, new_image_path)
    else:
        print(f"Image {filename} not found!")

    # Create JSON output for this file
    json_data = {"filename": filename, "score": rounded_score}


print("Processing complete.")


Processing complete.


In [14]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

# Example transforms, modify as needed
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset
datasets_path = 'Datasets'
full_dataset = datasets.ImageFolder(root=datasets_path, transform=transform)

def dataset_spliter(full_dataset, prozent_test=10, prozent_val=10, batch_size=32):
    if isinstance(prozent_test, int):
        prozent_test = prozent_test / 100
    if isinstance(prozent_val, int):
        prozent_val = prozent_val / 100
    
    # Calculate sizes for train, validation, and test
    test_size = int(prozent_test * len(full_dataset))
    val_size = int(prozent_val * len(full_dataset))
    train_size = len(full_dataset) - test_size - val_size

    # Split the dataset
    train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

# Example usage
train_loader, val_loader, test_loader = dataset_spliter(full_dataset=full_dataset, prozent_test=10, prozent_val=10, batch_size=32)
idx_to_class = {v: k for k, v in full_dataset.class_to_idx.items()}
num_classes = len(idx_to_class.keys())
class_names = list(idx_to_class.values())


In [76]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Chargement du modèle ResNet18 pré-entraîné
model = models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

# Only train the final layer
model.fc = nn.Linear(model.fc.in_features, len(class_names))
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)


model = model.to(device)

# Définition de la fonction de perte et de l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)



C:\Users\Sorel Tahata Djoumsi\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Sorel Tahata Djoumsi\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [77]:
from tqdm import tqdm

def train_model(model, train_loader, val_loader, num_epochs=20, learning_rate=0.001, patience=5):
    # Set device (GPU or CPU)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
    
    # Early stopping parameters
    best_val_loss = float('inf')
    epochs_no_improve = 0
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        # Training loop with tqdm progress bar
        with tqdm(train_loader, unit="batch") as tepoch:
            tepoch.set_description(f"Epoch {epoch+1}/{num_epochs}")
            
            for inputs, labels in tepoch:
                inputs, labels = inputs.to(device), labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                running_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
                # Update progress bar with current loss and accuracy
                tepoch.set_postfix(loss=loss.item(), accuracy=correct / total)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct / total
        
        # Validation loop
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        # Validation loop with tqdm progress bar
        with tqdm(val_loader, unit="batch", leave=False) as vepoch:
            vepoch.set_description(f"Validation")
            with torch.no_grad():
                for inputs, labels in vepoch:
                    inputs, labels = inputs.to(device), labels.to(device)

                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    
                    val_loss += loss.item() * inputs.size(0)
                    _, predicted = torch.max(outputs, 1)
                    val_total += labels.size(0)
                    val_correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct / val_total
        
        print(f'Epoch {epoch+1}/{num_epochs}, '
              f'Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, '
              f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
        
        # Step the scheduler
        scheduler.step(val_loss)
        
        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            best_model_state = model.state_dict()  # Save the best model
        else:
            epochs_no_improve += 1
        
        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            model.load_state_dict(best_model_state)  # Restore the best model
            break
        
    return model


def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    test_acc = correct / total
    print(f'Test Accuracy: {test_acc:.4f}')


model = train_model(model, train_loader, val_loader, num_epochs=50, patience=5)

# Test the model
test_model(model, test_loader)

torch.save(model.state_dict(), 'beauti_classificator.pth')

C:\Users\Sorel Tahata Djoumsi\AppData\Roaming\Python\Python312\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
Epoch 1/50: 100%|██████████| 138/138 [00:11<00:00, 11.87batch/s, accuracy=0.633, loss=0.977]


Epoch 1/50, Train Loss: 0.8966, Train Acc: 0.6332, Val Loss: 0.8219, Val Acc: 0.6600


Epoch 2/50: 100%|██████████| 138/138 [00:11<00:00, 11.85batch/s, accuracy=0.697, loss=0.63] 


Epoch 2/50, Train Loss: 0.7327, Train Acc: 0.6970, Val Loss: 0.7597, Val Acc: 0.6800


Epoch 3/50: 100%|██████████| 138/138 [00:11<00:00, 11.82batch/s, accuracy=0.704, loss=0.789]


...

Epoch 11/50, Train Loss: 0.6080, Train Acc: 0.7443, Val Loss: 0.6962, Val Acc: 0.7164


Epoch 12/50: 100%|██████████| 138/138 [00:11<00:00, 11.86batch/s, accuracy=0.747, loss=0.829]


Epoch 12/50, Train Loss: 0.6084, Train Acc: 0.7466, Val Loss: 0.6943, Val Acc: 0.7218


Epoch 13/50: 100%|██████████| 138/138 [00:11<00:00, 11.84batch/s, accuracy=0.763, loss=0.605]


Epoch 13/50, Train Loss: 0.5798, Train Acc: 0.7627, Val Loss: 0.6886, Val Acc: 0.7200
Early stopping triggered.
Test Accuracy: 0.7218
